# Time Filtering Demos

In [ ]:
from Ska.engarchive import fetch_eng as fetch
from Chandra.Time import DateTime

# This is added to allow plots to show up in the notebook
%matplotlib notebook

In [ ]:
tstart = '2015:018:00:00:00'
tstop = '2015:040:00:00:00'

I don't think you want to interpolate here (`eb1k5.interpolate()`). If you do it will automatically use a time sampling period of 32.8 seconds. This is useful when you have multiple MSIDs in the fetch object (via `fetch.MSIDSet`), but for a single MSID it is often best to stick to the original timeseries data.

In [ ]:
eb1k5 = fetch.Msid('eb1k5', tstart, tstop)
# eb1k5.interpolate()
eb1k5.iplot()

Here I use the `logical_intervals()` function built into the fetch object to eliminate the need for an extra library import. Because the object returned from this function is an Astropy Table, we can take advantage of the named column access to make the code a bit more readable to pull out the start and stop times for each K5=ON duration.

In [ ]:
eclipse_times = eb1k5.logical_intervals('==', 'ON ')
eclipse_times = zip(eclipse_times['datestart'], eclipse_times['datestop'])
eclipse_times

Next round each starting time to the start of the day, and the ending time to the end of the day.

In [ ]:
eclipse_times_rounded = []
for interval in eclipse_times:
    t1secs = DateTime('{}:00:00:00.000'.format(interval[0][:8])).secs
    t2secs = DateTime('{}:00:00:00.000'.format(interval[1][:8])).secs + 24*3600
    eclipse_times_rounded.append((DateTime(t1secs).date, DateTime(t2secs).date))

In [ ]:
eclipse_times_rounded

Fetch EPOWER1 and plot.

In [ ]:
epower1 = fetch.Msid('EPOWER1', tstart, tstop, stat='daily')
epower1.iplot()

Now use those times to filter out whole days from EPOWER1.

In [ ]:
epower1.remove_intervals(eclipse_times_rounded)
epower1.iplot()